<a href="https://colab.research.google.com/github/aknip/Coding-Cheatsheets/blob/main/Python-Compare-Text-or-JSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy python-benedict

In [4]:
import json
import textwrap

In [5]:
import numpy as np

def levenshtein(seq1, seq2):
    # source: https://github.com/vatsal220/medium_articles/blob/main/levenshtein_distance/lev_dist.ipynb
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    #print (matrix)
    return (matrix[size_x - 1, size_y - 1])

In [6]:
a = 'stamp'
b = 'stomp'
levenshtein(a,b)

1.0

In [7]:
json_target = {"Versicherungssumme": "5000000", "Versicherungsnehmer_Name": "Zorrsen Beteiligungsgesellschaft mbH", "test": {"testA": [{"key1": "value1", "key2": "value2"}, {"key3": "value3", "key4": "value4"}], "testB": 34}}
json_result = {"Versicherungsnehmer_Name": "Zorrsen Beteiligungsgesellschaft", "test": {'testB': 34, "testA": [{"key1": "value1", "key2": "value2"}, {"key3": "value3", "key4": "value4"}]}, "Versicherungssumme": "5500000"}

json_target_str = json.dumps(json_target, sort_keys = True)
json_result_str = json.dumps(json_result, sort_keys = True)

levenshtein(json_target_str,json_result_str)

5.0

# Idea: Create flat text-list from JSON

sort, then compare

In [12]:
from benedict import benedict

# source: https://stackoverflow.com/a/58611501
d = benedict({'dict1': {'foo': 1, 'bar': 2}, 'dict2': {'baz': 3, 'quux': 4}})
d = benedict(json_target)

def traverse_item(dct, key, value):
   #print('key: {} - value: {}'.format(key, value))
   print(str(key) + ': ' + str(value) + ': ' + str(type(value)) )

d.traverse(traverse_item)

Versicherungssumme: 5000000: <class 'str'>
Versicherungsnehmer_Name: Zorrsen Beteiligungsgesellschaft mbH: <class 'str'>
test: {'testA': [{'key1': 'value1', 'key2': 'value2'}, {'key3': 'value3', 'key4': 'value4'}], 'testB': 34}: <class 'benedict.dicts.benedict'>
testA: [{'key1': 'value1', 'key2': 'value2'}, {'key3': 'value3', 'key4': 'value4'}]: <class 'list'>
0: {'key1': 'value1', 'key2': 'value2'}: <class 'benedict.dicts.benedict'>
key1: value1: <class 'str'>
key2: value2: <class 'str'>
1: {'key3': 'value3', 'key4': 'value4'}: <class 'benedict.dicts.benedict'>
key3: value3: <class 'str'>
key4: value4: <class 'str'>
testB: 34: <class 'int'>


In [ ]:
json.loads()
json.dumps(response_JSON_Obj, indent=4)